In [0]:
# Databricks notebook source
"""
# 03_gold_analytics
Descrição:
    Gera visões de negócio agregadas a partir da camada Silver.
    Filtra apenas os clientes ativos (is_current = true) para contabilização atual.
"""

from pyspark.sql.functions import *

In [0]:
# 1. Configuração
source_table = "main.default.silver_customers"
target_table_name = "main.default.gold_customers_by_city"

In [0]:
# 2. Leitura da Silver (Snapshot Atual)
# Importante: Na Gold, geralmente queremos ver o "estado atual", então filtramos is_current
df_silver = spark.read.table(source_table).filter("is_current = true")


In [0]:
# 3. Agregação (Lógica de Negócio)
# KPI: Contagem de clientes ativos por cidade e data de referência
df_gold = (df_silver
    .groupBy("city")
    .agg(
        count("customer_id").alias("active_customers_count"),
        max("update_ts").alias("last_updated_ts")
    )
    .withColumn("ref_date", current_date())
)

In [0]:
# 4. Escrita (Overwrite ou Append)
# Para tabelas de snapshot diário, costumamos usar Overwrite (substitui a visão do dia)
# ou Append particionado por data. Vamos de Overwrite para simplificar o dashboard.

(df_gold.write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    # REMOVIDO: .option("path", ...) -> Tabela Gerenciada
    .saveAsTable(target_table_name)
)

print(f"✅ Tabela Gold '{target_table_name}' atualizada!")
display(df_gold)